In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import ViTModel, ViTImageProcessor, RobertaModel, AutoTokenizer
from torchvision import transforms
from PIL import Image
import os

# Define Dataset Class
class VQADataset(Dataset):
    def __init__(self, data, label2idx, img_feature_extractor, text_tokenizer, device, transforms=None, img_dir='images/'):
        self.data = data
        self.img_dir = img_dir
        self.label2idx = label2idx
        self.img_feature_extractor = img_feature_extractor
        self.text_tokenizer = text_tokenizer
        self.device = device
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path = os.path.join(self.img_dir, self.data[index]['image_path'])
        img = Image.open(img_path).convert('RGB')

        if self.transforms:
            img = self.transforms(img)

        img = self.img_feature_extractor(images=img, return_tensors="pt")
        img = {k: v.to(self.device).squeeze(0) for k, v in img.items()}

        question = self.data[index]['question']
        question = self.text_tokenizer(question, padding="max_length", max_length=20, truncation=True, return_tensors="pt")
        question = {k: v.to(self.device).squeeze(0) for k, v in question.items()}

        label = torch.tensor(self.label2idx[self.data[index]['answer']], dtype=torch.long).to(self.device)

        return {'image': img, 'question': question, 'label': label}

# Define Model Components
class VisualEncoder(nn.Module):
    def __init__(self):
        super(VisualEncoder, self).__init__()
        self.model = ViTModel.from_pretrained("google/vit-base-patch16-224")

    def forward(self, inputs):
        outputs = self.model(**inputs)
        return outputs.pooler_output

class TextEncoder(nn.Module):
    def __init__(self):
        super(TextEncoder, self).__init__()
        self.model = RobertaModel.from_pretrained("roberta-base")

    def forward(self, inputs):
        outputs = self.model(**inputs)
        return outputs.pooler_output

class Classifier(nn.Module):
    def __init__(self, hidden_size=512, dropout_prob=0.2, n_classes=2):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(768 * 2, hidden_size)
        self.dropout = nn.Dropout(dropout_prob)
        self.gelu = nn.GELU()
        self.fc2 = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class VQAModel(nn.Module):
    def __init__(self, visual_encoder, text_encoder, classifier):
        super(VQAModel, self).__init__()
        self.visual_encoder = visual_encoder
        self.text_encoder = text_encoder
        self.classifier = classifier

    def forward(self, image, question):
        image_out = self.visual_encoder(image)
        text_out = self.text_encoder(question)
        x = torch.cat((image_out, text_out), dim=1)
        return self.classifier(x)

# Define Training and Evaluation Functions
def evaluate(model, dataloader, criterion):
    model.eval()
    correct, total, losses = 0, 0, []
    with torch.no_grad():
        for inputs in dataloader:
            images, questions, labels = inputs['image'], inputs['question'], inputs['label']
            outputs = model(images, questions)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return sum(losses) / len(losses), correct / total

def train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs):
    for epoch in range(epochs):
        model.train()
        batch_losses = []
        for inputs in train_loader:
            images, questions, labels = inputs['image'], inputs['question'], inputs['label']
            optimizer.zero_grad()
            outputs = model(images, questions)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())
        val_loss, val_acc = evaluate(model, val_loader, criterion)
        scheduler.step()
        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {sum(batch_losses) / len(batch_losses):.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# Main Execution
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Data Preparation
    img_feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
    text_tokenizer = AutoTokenizer.from_pretrained("roberta-base")

    train_data, val_data, test_data = [], [], []  # Load your data here
    label2idx = {'yes': 0, 'no': 1}  # Adjust based on your dataset

    data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = VQADataset(train_data, label2idx, img_feature_extractor, text_tokenizer, device, data_transform)
    val_dataset = VQADataset(val_data, label2idx, img_feature_extractor, text_tokenizer, device, data_transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Model Initialization
    visual_encoder = VisualEncoder().to(device)
    text_encoder = TextEncoder().to(device)
    classifier = Classifier().to(device)
    model = VQAModel(visual_encoder, text_encoder, classifier).to(device)

    # Training Setup
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10)
